In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
import os

In [6]:
os.listdir('./moviedata')

['credits.csv',
 'keywords.csv',
 'links.csv',
 'links_small.csv',
 'movies_metadata.csv',
 'ratings.csv',
 'ratings_small.csv']

In [7]:
data = pd.read_csv('./moviedata/movies_metadata.csv', low_memory=False)
display(data.head(3))

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [8]:
data = data.head(20000)
data

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://www.lordreetlamorale-lefilm.com/#/nav/t...,76609,tt1242521,fr,L'Ordre et la Morale,Dissidents in a French colony attack a police ...,...,2011-11-16,0.0,136.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Rebellion,False,6.3,24.0
19996,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,8941,tt1132607,fr,Versailles,A young mother Nina and her son Enzo find them...,...,2008-05-19,0.0,113.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Versailles,False,6.0,4.0
19997,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}, {'id': 107...",NaN,64074,tt1528224,en,Two in the Wave,An in-depth analysis of the relationship betwe...,...,2010-05-19,0.0,91.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,NaN,Two in the Wave,False,5.5,5.0
19998,False,NaN,0,[],NaN,197723,tt0471239,en,Lotte Reiniger: Homage to the Inventor of the ...,Follows the life and work of animator Lotte Re...,...,2001-11-19,0.0,0.0,[],Released,NaN,Lotte Reiniger: Homage to the Inventor of the ...,False,0.0,0.0


In [9]:
data['overview'].isnull().sum()

135

In [10]:
data['overview'] = data['overview'].fillna('')

In [11]:
# overview 열에 대한 TF-IDF

tfidf = TfidfVectorizer(stop_words ='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
tfidf_matrix.shape

(20000, 47487)

    - 20,000개의 행과 47,487개의 열을 가짐
    => 20,000개의 영화를 표현하기 위해 총 47,487개의 단어가 사용됨
    혹은 47,487차원의 문서 벡터 20,000개가 존재함

In [13]:
# 20,000개의 문서 벡터에 대해 상호 간의 코사인 유사도 구하기

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(20000, 20000)

    - 20,000개의 문서 벡터(영화 줄거리벡터) 와 자기 자신을 포함한 20,000개의 문서 벡터간 유사도가 기록됨

In [14]:
title_to_idx = dict(zip(data['title'], data.index))
title_to_idx

{'Toy Story': 0,
 'Jumanji': 1,
 'Grumpier Old Men': 2,
 'Waiting to Exhale': 3,
 'Father of the Bride Part II': 4,
 'Heat': 14684,
 'Sabrina': 888,
 'Tom and Huck': 7,
 'Sudden Death': 8,
 'GoldenEye': 9,
 'The American President': 10,
 'Dracula: Dead and Loving It': 11,
 'Balto': 12,
 'Nixon': 13,
 'Cutthroat Island': 14,
 'Casino': 15,
 'Sense and Sensibility': 16,
 'Four Rooms': 17,
 'Ace Ventura: When Nature Calls': 18,
 'Money Train': 19,
 'Get Shorty': 20,
 'Copycat': 21,
 'Assassins': 22,
 'Powder': 23,
 'Leaving Las Vegas': 24,
 'Othello': 15347,
 'Now and Then': 26,
 'Persuasion': 14838,
 'The City of Lost Children': 28,
 'Shanghai Triad': 29,
 'Dangerous Minds': 30,
 'Twelve Monkeys': 31,
 'Wings of Courage': 32,
 'Babe': 33,
 'Carrington': 34,
 'Dead Man Walking': 35,
 'Across the Sea of Time': 36,
 'It Takes Two': 37,
 'Clueless': 38,
 'Cry, the Beloved Country': 39,
 'Richard III': 17719,
 'Dead Presidents': 41,
 'Restoration': 42,
 'Mortal Kombat': 43,
 'To Die For': 44,

In [23]:
# Father of the Bride Part 2 idx ?

title_to_idx['Father of the Bride Part II']

4

In [35]:
# 선택한 영화 제목을 입력하면 코사인 유사도를 통해 overview가 가장 유사한 10개를 찾아냄

def get_recommendation(title, cosine_sim=cosine_sim):
    idx = title_to_idx[title]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]

    movie_indices = [idx[0] for idx in sim_scores]
    
    return data['title'].iloc[movie_indices]

    - method 프로세스 예시

In [25]:
idx = title_to_idx['Toy Story']
idx

0

In [27]:
sim_scores = list(enumerate(cosine_sim[0]))
sim_scores

[(0, 1.0),
 (1, 0.015757477316785385),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.04113995020790759),
 (18, 0.0),
 (19, 0.0),
 (20, 0.009940909498428008),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.01983079545199659),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0063477440216163795),
 (43, 0.0),
 (44, 0.0),
 (45, 0.009320555328259442),
 (46, 0.0),
 (47, 0.0),
 (48, 0.0),
 (49, 0.013821954288971792),
 (50, 0.00985720337272072),
 (51, 0.010960042528831588),
 (52, 0.0),
 (53, 0.0),
 (54, 0.019958825373755495),
 (55, 0.0),
 (56, 0.02534679843095959),
 (57, 0.020764246377724647),
 (58, 0.0),
 (59, 0.03424230329305141),
 (60, 0.0),
 (61, 0.0),
 (62, 0.008609788740840691),
 (63, 0.0),
 (64, 0.

In [28]:
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores

[(0, 1.0),
 (15348, 0.5258229300737997),
 (2997, 0.462638821864108),
 (10301, 0.2795541150488133),
 (8327, 0.20068589066985557),
 (1071, 0.1827296872275654),
 (11399, 0.15689199223985253),
 (1932, 0.15273436907628185),
 (3057, 0.14319886672358997),
 (485, 0.13751067912693815),
 (11606, 0.133606616102742),
 (17189, 0.1328092314940444),
 (7254, 0.12967905136381996),
 (5797, 0.1291947019352676),
 (13947, 0.12279774665302924),
 (1032, 0.12186692256654189),
 (3252, 0.1207957695607309),
 (2157, 0.12016025669738598),
 (7615, 0.1196946828667064),
 (6944, 0.1196303925557605),
 (19976, 0.11798196632714826),
 (5618, 0.11789897140512816),
 (4078, 0.11785943136209667),
 (448, 0.11656473849605974),
 (10585, 0.11096862216325075),
 (6435, 0.11092030773121374),
 (3145, 0.10730566352936693),
 (15169, 0.1040535956808567),
 (12072, 0.10398823916088912),
 (2635, 0.09704920510483701),
 (10375, 0.09487165910487882),
 (9963, 0.09405545388086307),
 (16991, 0.09106345606213505),
 (16692, 0.08743114134258186),
 

In [29]:
sim_scores = sim_scores[1:11]
sim_scores

[(15348, 0.5258229300737997),
 (2997, 0.462638821864108),
 (10301, 0.2795541150488133),
 (8327, 0.20068589066985557),
 (1071, 0.1827296872275654),
 (11399, 0.15689199223985253),
 (1932, 0.15273436907628185),
 (3057, 0.14319886672358997),
 (485, 0.13751067912693815),
 (11606, 0.133606616102742)]

In [36]:
movie_indices = [idx[0] for idx in sim_scores]
movie_indices

[15348, 2997, 10301, 8327, 1071, 11399, 1932, 3057, 485, 11606]

In [37]:
data['title'].iloc[min_indices]

15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
11399    For Your Consideration
1932                  Condorman
3057            Man on the Moon
485                      Malice
11606              Factory Girl
Name: title, dtype: object

In [38]:
get_recommendation('Toy Story')

15348               Toy Story 3
2997                Toy Story 2
10301    The 40 Year Old Virgin
8327                  The Champ
1071      Rebel Without a Cause
11399    For Your Consideration
1932                  Condorman
3057            Man on the Moon
485                      Malice
11606              Factory Girl
Name: title, dtype: object